# Financial Instruments - Homework 6

## 1. Implied Volatility

**Goal:** Calculate IV for ATM option on Feb 12, 2024.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

# Data
df_opts = pd.read_excel("Assignments/Assignment 6/QuoteData_2024 (1).xls", header=3)
df_opts.columns = [c.strip() for c in df_opts.columns]

S0 = 5022.0
T = 130.0 / 365.0
r = 0.054
q = 0.015

def bs_price(sigma, S, K, T, r, q, type='call'):
    d1 = (np.log(S/K) + (r - q + 0.5 * sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if type == 'call':
        return S * np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    else:
        return K * np.exp(-r*T) * norm.cdf(-d2) - S * np.exp(-q*T) * norm.cdf(-d1)

def get_iv(price, K, type='call'):
    try:
        return brentq(lambda s: bs_price(s, S0, K, T, r, q, type) - price, 0.001, 2.0)
    except:
        return np.nan

# ATM Option
df_opts['Diff'] = abs(df_opts['Strike'] - S0)
atm_row = df_opts.loc[df_opts['Diff'].idxmin()]
K_atm = atm_row['Strike']
C_mid = (atm_row['Bid'] + atm_row['Ask']) / 2
iv_atm = get_iv(C_mid, K_atm)

print(f"Spot: {S0}")
print(f"ATM Strike: {K_atm}")
print(f"ATM Price: {C_mid}")
print(f"Implied Volatility: {iv_atm:.4%}")

## 2. Morgan Stanley PLUS Valuation

In [ ]:
Principal = 10.0
K1 = S0
K2 = S0 * (1 + 0.19/3.0)
T_plus = 1.0
sigma_plus = iv_atm

val_stock = 10.0 * np.exp(-q * T_plus)
c1 = bs_price(sigma_plus, S0, K1, T_plus, r, q, 'call')
val_c1 = (20.0 / S0) * c1
c2 = bs_price(sigma_plus, S0, K2, T_plus, r, q, 'call')
val_c2 = -(30.0 / S0) * c2

val_total = val_stock + val_c1 + val_c2
print(f"Fair Value: {val_total:.4f}")